In [1]:
import pandas as pd



In [2]:
mps = pd.read_csv("data/renamed-mps.csv")

In [3]:
mps

,picture,name,birth_date,birth_city,birth_province_region,birth_country,deceased_date,role_type_of_parliamentarian,seat_riding_senatorial_division,seat_province_territory,gender,profession,political_affiliation,date_of_parliamentary_entry,role_minister,role_critic,years_of_service,military_service
0,NaN,"Abbott, Anthony Chisholm",1930-11-26,Montreal,Quebec,Canada,NaN,MP (1974/07/08 - 1979/05/21),Mississauga (1974/07/08 - 1979/05/21),Ontario,M,lawyer,Liberal Party of Canada (1974/07/08 - 1979/05/21),1974-07-08,Minister of National Revenue (1978/11/24 - 197...,NaN,"1778 days (4 years, 10 months, 13 days)",NaN
1,NaN,"Abbott, Douglas Charles",1899-05-29,Lennoxville,Quebec,Canada,1987-03-15,MP (1953/08/10 - 1954/06/30)MP (1949/06/27 - 1...,Saint-Antoine--Westmount (1953/08/10 - 1954/06...,QuebecQuebecQuebec,M,lawyer,Liberal Party of Canada (1940/03/26 - 1954/06/30),1940-03-26,Minister of Finance and Receiver General (1946...,NaN,"5207 days (14 years, 3 months, 3 days)",NaN
2,NaN,"Abbott, Jim",1942-08-18,Toronto,Ontario,Canada,NaN,MP (1997/06/02 - 2011/05/01)MP (1993/10/25 - 1...,Kootenay--Columbia (1997/06/02 - 2011/05/01)Ko...,British ColumbiaBritish Columbia,M,"manager, businessman",Conservative Party of Canada (2003/12/23 - 201...,1993-10-25,NaN,Canadian Heritage (2001/06/18 - 2004/07/21)Spo...,"6396 days (17 years, 6 months, 4 days)",NaN
3,NaN,"Abbott, John Joseph Caldwell",1821-03-12,St. Andrews,,Lower Canada,1893-10-30 (Died in Office),MP (1881/08/17 - 1887/02/21)MP (1880/02/12 - 1...,Inkerman (1887/05/12 - 1893/10/30)Argenteuil (...,QuebecQuebecQuebecQuebec,M,"professor, lawyer, dean",Liberal-Conservative (1887/05/12 - 1893/10/30)...,1867-09-20,President of the Privy Council (1891/06/16 - 1...,NaN,"7467 days (20 years, 5 months, 10 days)",NaN
4,NaN,"Ablonczy, Diane",1949-05-06,Peoria,Illinois,United States,NaN,MP (1997/06/02 - 2015/10/18)MP (1993/10/25 - 1...,Calgary Nose Hill (1997/06/02 - 2015/10/18)Cal...,AlbertaAlberta,F,"farmer, lawyer, teacher",Conservative Party of Canada (2003/12/23 - 201...,1993-10-25,NaN,Citizenship and Immigration (2002/04/04 - 2006...,"8027 days (21 years, 11 months, 22 days)",NaN
5,NaN,"Aboultaif, Ziad",1966,Rachaya,Beqaa,Lebanon,NaN,MP (2015/10/19 - ),Edmonton Manning (2015/10/19 - ),Alberta,M,business owner,Conservative Party of Canada (2015/10/19 - ),2015-10-19,NaN,International Development (2017/08/30 - )Natio...,"995 days (2 years, 8 months, 22 days)",NaN
6,NaN,"Achim, Honoré",1881-08-02,Montreal,Quebec,Canada,1950-05-14,MP (1911/09/21 - 1917/12/16),Labelle (1911/09/21 - 1917/12/16),Quebec,M,lawyer,Liberal Party of Canada (1917/06/29 - 1917/12/...,1911-09-21,NaN,NaN,"2278 days (6 years, 2 months, 26 days)",NaN
7,NaN,"Adams, Eve",1974-11-07,Sudbury,Ontario,Canada,NaN,MP (2011/05/02 - 2015/10/18),Mississauga--Brampton South (2011/05/02 - 2015...,Ontario,F,"public servant, community activist",Liberal Party of Canada (2015/02/09 - 2015/10/...,2011-05-02,NaN,NaN,"1630 days (4 years, 5 months, 17 days)",NaN
8,NaN,"Adams, Michael",1845-08-13,Douglastown,New Brunswick,Canada,1899-01-02,MP (1891/03/05 - 1896/01/06)Senator (1896/01/0...,Northumberland (1896/01/07 - 1899/01/02)Northu...,New BrunswickNew Brunswick,M,lawyer,Conservative (1867-1942) (1891/03/05 - 1899/01...,1891-03-05,NaN,NaN,"2859 days (7 years, 9 months, 28 days)",NaN
9,NaN,"Adams, Peter",1936-04-17,,,United Kingdom,NaN,MP (1993/10/25 - 2006/01/22),Peterborough (1993/10/25 - 2006/01/22),Ontario,M,professor,Liberal Party of Canada (1993/10/25 - 2006/01/22),1993-10-25,NaN,NaN,"4472 days (12 years, 2 months, 28 days)",NaN


Conversion notes

* Create a second table for manipulation—it’ll keep just one row per MP, but better structure the data within
* Collect [all the date-bound variables](https://github.com/lchski/mps-analysis/issues/10), but separate them into arrays in each column (instead of just mashed-together text)
* Using the `role_type_of_parliamentarian` column, assemble the first table with a row for each day of service. For each day, pull from the date-bound variables, finding the value with a time range that fits

e.g. for Jim Abbott (third in dataset):

* Jim
  * `role_type_of_parliamentarian`
      * MP (1997/06/02 - 2011/05/01)
      * MP (1993/10/25 - 1997/06/01)
  * `seat_riding_senatorial_division`
      * Kootenay--Columbia (1997/06/02 - 2011/05/01)
      * Kootenay East (1993/10/25 - 1997/06/01)
  * `political_affiliation`
      * Conservative Party of Canada (2003/12/23 - 2011/05/01)
      * Canadian Reform Conservative Alliance (2000/03/27 - 2003/12/22)
      * Reform Party (1993/10/25 - 2000/03/26)
  * `role_minister` (n/a)
  * `role_critic`
      * Canadian Heritage (2001/06/18 - 2004/07/21)
      * Sport (2003/06/20 - 2004/01/21)
      * Amateur Sport (2002/02/01 - 2003/06/19)
      * Solicitor General (1998/06/16 - 2000/07/31)
      * Canadian Heritage (1996 - 1998/06/16)
      * Leader of the Government in the Senate (1996 - 1997/06/19)
      * Agriculture and Agri-Food (1995/1 - 1996)
      * Senate (1995 - 1996)
      * Regional Development (1995 - 1996)
      * Energy (1994 - 1996)
      * National Revenue (1994 - 1995)
      * Environment (1994 - 1995)
      * Fisheries and Oceans (1994 - 1995)

In [8]:
mps.set_index(['name', 'birth_date'], inplace=True)

In [37]:
def clean_role(role):
    ## split into each period of service
    roles = role.split(")")
    
    ## split into type of service and period of service
    roles = list(map(lambda x : x.split("("), roles))
    
    ## remove extra whitespace
    roles = list(map(lambda x : list(map(lambda y : y.strip(), x)), roles))
    
    ## exclude empty [''] objects accidentally created from our operations
    roles = list(filter(lambda x : x != [''], roles))
    
    ## woo, we made it!
    return roles

print(clean_role("MP (1974/07/08 - 1979/05/21)"))
print(clean_role("MP (1997/06/02 - 2011/05/01)MP (1993/10/25 - 1997/06/01)"))

[['MP', '1974/07/08 - 1979/05/21']]
[['MP', '1997/06/02 - 2011/05/01'], ['MP', '1993/10/25 - 1997/06/01']]


Object `assert` not found.
